In [3]:
!pip install snowflake-connector-python pandas scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 7.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import snowflake.connector

# Snowflake connection details
def get_snowflake_connection():
    return snowflake.connector.connect(
        user='connorjfq',
        password='Group_project1',
        account='GTNADPW-LHB58441',
        warehouse='PROJECT_WH',
        database='PROJECT_DB',
        schema='RAW_DATA_SCHEMA'
    )

# Step 1: Fetch data from Snowflake
def fetch_data_from_snowflake():
    conn = get_snowflake_connection()
    cursor = conn.cursor()

    # Check if the table exists, create if not
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS PROJECT_DB.ANALYTICS.PREDICTIONS (
            DATE DATE,
            LAT FLOAT,
            LONG FLOAT,
            CURRENT_SPEED FLOAT,
            PREDICTED_SPEED FLOAT,
            IS_CONGESTED BOOLEAN
        );
    """)

    query = """
        SELECT DATE, LAT, LONG, CURRENT_SPEED, FREE_SPEED
        FROM TRAFFIC_DATA;
    """
    cursor.execute(query)
    rows = cursor.fetchall()

    # Convert to DataFrame
    df = pd.DataFrame(rows, columns=["DATE", "LAT", "LONG", "CURRENT_SPEED", "FREE_SPEED"])
    cursor.close()
    conn.close()
    return df

# Step 2: Train Linear Regression model, make predictions, and determine congestion
def train_and_predict(df):
    if df.empty:
        print("No data available for the selected time range. Skipping predictions.")
        return None

    # Feature engineering
    df["normalized_speed"] = df["FREE_SPEED"] / df["FREE_SPEED"].min()
    X = df[["LAT", "LONG", "FREE_SPEED", "normalized_speed"]]
    y = df["CURRENT_SPEED"]

    # Train model
    model = LinearRegression()
    model.fit(X, y)

    # Make predictions and assess congestion
    df["PREDICTED_SPEED"] = model.predict(X)
    df["IS_CONGESTED"] = df["PREDICTED_SPEED"] < 0.8 * df["FREE_SPEED"]
    return df

# Step 3: Insert predictions back into Snowflake
def insert_predictions_to_snowflake(df):
    conn = get_snowflake_connection()
    cursor = conn.cursor()

    # Convert DATE column to string in Snowflake-compatible format
    df["DATE"] = df["DATE"].dt.strftime("%Y-%m-%d")

    insert_query = """
        INSERT INTO PROJECT_DB.ANALYTICS.PREDICTIONS (DATE, LAT, LONG, CURRENT_SPEED, PREDICTED_SPEED, IS_CONGESTED)
        VALUES (%s, %s, %s, %s, %s, %s);
    """
    for _, row in df.iterrows():
        cursor.execute(insert_query, tuple(row[["DATE", "LAT", "LONG", "CURRENT_SPEED", "PREDICTED_SPEED", "IS_CONGESTED"]]))
    conn.commit()
    cursor.close()
    conn.close()

# Main workflow
def main():
    print("Fetching data from Snowflake...")
    data = fetch_data_from_snowflake()

    if data.empty:
        print("No data fetched. Exiting.")
        return

    print("Training model and making predictions...")
    predictions = train_and_predict(data)

    if predictions is not None:
        print("Uploading predictions to Snowflake...")
        insert_predictions_to_snowflake(predictions)
        print("Predictions uploaded successfully.")

# Run the main function
main()


Fetching data from Snowflake...
Training model and making predictions...
Uploading predictions to Snowflake...
